In [1]:
from dataset import Cifar100
from model import ClassificationModel
from utils import train_classification

import torch
import torch.nn as nn
import torchvision.transforms.functional as F
from torchvision import  transforms
import matplotlib.pyplot as plt
import numpy as np

import random

In [2]:
model = torch.load('ae250.model').cuda()
class_model = ClassificationModel().cuda()
loss_function = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(class_model.parameters(), lr=5e-3)
class_model

ClassificationModel(
  (conv_block): Sequential(
    (0): Conv2d(3, 8, kernel_size=(5, 5), stride=(3, 3))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout2d(p=0.5, inplace=False)
    (4): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): Dropout2d(p=0.5, inplace=False)
  )
  (fc_block): Sequential(
    (0): Linear(in_features=1600, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=1024, out_features=100, bias=True)
    (4): LogSoftmax(dim=1)
  )
)

In [3]:
train_data = Cifar100("CIFAR100/TRAIN")
test_data = Cifar100("CIFAR100/TEST")

train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=128, shuffle=True, num_workers=12
)

test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=128, num_workers=12
)

In [4]:
transform = transforms.RandomApply(
    torch.nn.ModuleList([
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=(0, 90))
    ]), p=0.5)

In [23]:
def plot_result(train_losses, train_accuracies, test_accuracies):
    x = list(range(len(train_losses)))

    f, polts = plt.subplots(1, 2)
    f.set_figheight(5)
    f.set_figwidth(15)

    polts[0].plot(x, train_accuracies)
    polts[0].plot(x, test_accuracies)
    polts[0].legend(["train accuracies", "test accuracies"])

    polts[1].plot(x, train_losses)
    polts[1].legend(["train loss"])

    plt.show()

In [ ]:
train_losses, train_accuracies, test_accuracies = train_classification(
    10, model, class_model, train_loader, test_loader, loss_function, optimizer, 
    transform=transform, noti_rate=1000
)

plot_result(train_losses, train_accuracies, test_accuracies)

In [7]:
for i in train_loader:
    break
classes = i[1].cuda()
i = i[0] / 255
i = i.cuda()

In [8]:
classes

tensor([29, 62, 66, 48, 84, 38, 55, 89, 23, 68, 24,  3, 34, 60, 93, 98, 97, 63,
        93, 31, 24, 43,  1, 83, 21, 98, 45,  2, 95, 51, 11, 59, 94, 29,  8, 61,
        54, 51, 36, 87, 39,  9, 23, 38, 79, 95, 78, 93, 10, 42, 62, 83, 44, 83,
         6, 45, 74, 66, 32, 96, 98, 10, 14, 65, 57, 91,  9, 27, 74, 95, 14, 40,
        70, 42, 22,  5, 82, 97, 70, 61, 73, 31, 18, 98, 38, 17, 50, 24, 36, 15,
        29, 74, 84, 77, 53, 44, 63, 26, 38, 38, 20,  2, 88, 38, 45, 72, 83, 84,
        78, 17, 24, 95, 93,  0, 59,  0, 97, 27, 33, 51, 14,  4, 12, 15, 94, 14,
        54, 47], device='cuda:0')

In [9]:
out = class_model(model(i))
out.shape

torch.Size([128, 100])

In [10]:
out.argmax(1)

tensor([29, 14, 67, 20, 14, 11, 65, 22, 16, 12, 35, 37, 75, 23, 23, 40, 75, 14,
        75, 31, 24, 60, 42, 11, 21, 73, 24, 22, 11, 23, 49, 57, 38, 94,  8, 26,
        54,  1, 36, 60, 39,  9, 22, 11, 99, 78, 37, 60, 69, 42, 60, 88, 74, 73,
        79, 24, 74, 22, 24, 76, 21, 89,  7, 23, 57, 74, 19, 73, 74, 18, 22, 34,
        52, 42,  5, 41, 94, 19, 24, 49, 75,  1, 29, 21, 38, 11, 50, 24, 60, 75,
        23, 74, 14, 92, 20, 23, 73, 94, 69, 93, 23, 80, 11, 11, 21, 80, 44, 59,
         3, 75, 24, 22, 21, 22, 94, 75, 60, 52, 74, 77, 14, 77, 12, 88, 94, 14,
        26, 49], device='cuda:0')

In [11]:
loss_function(out, classes)

tensor(3.9219, device='cuda:0', grad_fn=<NllLossBackward0>)

In [12]:
sum(out.argmax(1) == classes).item()

23